In [1]:
import time

import torch
import torchaudio
from datasets import load_dataset
from IPython.display import Audio
from huggingface_hub import hf_hub_download

from sam_audio import SAMAudio, SAMAudioProcessor


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/imagebind/data.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Create a synthetic 8-second audio clip with a horn-like tone around the anchor window
sample_rate = 48_000
duration = 8.0
num_samples = int(sample_rate * duration)
# Base noise floor
wav = 0.01 * torch.randn(num_samples)
# Add a horn-like sine burst between 6.3s and 7.0s
start = int(6.3 * sample_rate)
end = int(7.0 * sample_rate)
t = torch.linspace(0, (end - start) / sample_rate, end - start, dtype=torch.float32)
wav[start:end] += 0.2 * torch.sin(2 * torch.pi * 220 * t)
# Shape to (channels, time)
samples = wav.unsqueeze(0)
Audio(wav.numpy(), rate=sample_rate)


In [3]:
model_id = "facebook/sam-audio-small"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading {model_id} on {device}...")

# Disable optional components that can trigger extra checkpoint downloads and slow startup
# (rankers + span predictor are not needed for this notebook)
t0 = time.time()
model = SAMAudio.from_pretrained(
    model_id,
    visual_ranker=None,
    text_ranker=None,
    span_predictor=None,
).to(device).eval()
processor = SAMAudioProcessor.from_pretrained(model_id)
print(f"Loaded in {time.time() - t0:.1f}s")


Loading facebook/sam-audio-small on cuda...


Fetching 5 files:   0%|                                                                                                                                 | 0/5 [00:00<?, ?it/s]

Fetching 5 files:  20%|████████████████████████▏                                                                                                | 1/5 [00:00<00:01,  3.09it/s]

Fetching 5 files:  80%|████████████████████████████████████████████████████████████████████████████████████████████████▊                        | 4/5 [03:43<01:00, 60.31s/it]

Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:43<00:00, 44.65s/it]


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded in 274.8s


In [4]:
wav = torchaudio.functional.resample(
    samples, sample_rate, processor.audio_sampling_rate
)
wav = wav.mean(0, keepdim=True)
inputs = processor(
    audios=[wav], descriptions=["A horn honking"], anchors=[[ ["+", 6.3, 7.0] ]]
).to(device)

t0 = time.time()
with torch.inference_mode():
    result = model.separate(
        inputs,
        ode_opt={"method": "midpoint", "options": {"step_size": 2 / 8}},
    )
print(f"Separation finished in {time.time() - t0:.1f}s")


Separation finished in 9.2s


In [5]:
Audio(result.target[0].cpu(), rate=processor.audio_sampling_rate)